In [1]:
%matplotlib inline

from spacepy import pycdf
import numpy as np
import datetime
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection 
from matplotlib.colors import ListedColormap, BoundaryNorm
import matplotlib.cm as pltcm
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import pickle


In [2]:
# Topics: line, color, LineCollection, cmap, colorline, codex
'''
Defines a function colorline that draws a (multi-)colored 2D line with coordinates x and y.
The color is taken from optional data in z, and creates a LineCollection.

z can be:
- empty, in which case a default coloring will be used based on the position along the input arrays
- a single number, for a uniform color [this can also be accomplished with the usual plt.plot]
- an array of the length of at least the same length as x, to color according to this data
- an array of a smaller length, in which case the colors are repeated along the curve

The function colorline returns the LineCollection created, which can be modified afterwards.

See also: plt.streamplot
'''

# Data manipulation:

def make_segments(x, y):
    '''
    Create list of line segments from x and y coordinates, in the correct format for LineCollection:
    an array of the form   numlines x (points per line) x 2 (x and y) array
    '''

    points = np.array([x, y]).T.reshape(-1, 1, 2)
    segments = np.concatenate([points[:-1], points[1:]], axis=1)
    
    return segments


# Interface to LineCollection:

def colorline(x, y, z=None, cmap=plt.get_cmap('copper'), norm=plt.Normalize(0.0, 1.0), linewidth=3, alpha=1.0):
    '''
    Plot a colored line with coordinates x and y
    Optionally specify colors in the array z
    Optionally specify a colormap, a norm function and a line width
    '''
    
    # Default colors equally spaced on [0,1]:
    if z is None:
        z = np.logspace(1, 5, len(x))
           
    # Special case if a single number:
    if not hasattr(z, "__iter__"):  # to check for numerical input -- this is a hack
        z = np.array([z])
        
    z = np.asarray(z)
    
    segments = make_segments(x, y)
    lc = LineCollection(segments, array=z, cmap=cmap, norm=norm, linewidth=linewidth, alpha=alpha)
    
    ax = plt.gca()
    ax.add_collection(lc)
    
    return lc
        
    
def clear_frame(ax=None): 
    # Taken from a post by Tony S Yu
    if ax is None: 
        ax = plt.gca() 
    ax.xaxis.set_visible(False) 
    ax.yaxis.set_visible(False) 
    for spine in ax.spines.itervalues(): 
        spine.set_visible(False) 

In [3]:

def plot_colorline(x,y,c):
    c = pltcm.plasma((c-np.min(c))/(np.max(c)-np.min(c)))
    ax = plt.gca()
    for i in np.arange(len(x)-1):
        ax.plot([x[i],x[i+1]], [y[i],y[i+1]], c=c[i], lw=5)
        print ("IDX: {}".format(i))
    return

In [4]:
path="/Volumes/8TB Seagate/PhD Data/RBSPcdf/"
pathout="/Volumes/8TB Seagate/PhD Data/profiles/"

In [5]:
def get_col(x,y,t):
    points = np.array([x, y]).T.reshape(-1, 1, 2)
    segments = np.concatenate([points[:-1], points[1:]], axis=1)
    lc = LineCollection(segments, cmap=plt.get_cmap('gist_stern'),
                        norm=colors.LogNorm(1e-0, 1e3))
    lc.set_array(t)
    lc.set_linewidth(3)
    return lc


In [6]:
# ephemA = pycdf.CDF(path+"rbsp-a_magnetometer_hires-sm_emfisis-l3_20131002_v1.3.4.cdf")
# ephemB = pycdf.CDF(path+"rbsp-b_magnetometer_hires-sm_emfisis-l3_20131002_v1.6.1.cdf")

In [7]:

pts_EPOCH_a = pickle.load(open("./smEpochA.pickle", "rb"))
pts_EPOCH_b = pickle.load(open("./smEpochB.pickle", "rb"))

pts_sm_a = pickle.load(open("./smPositionsRBSP_A.pickle", "rb"))
pts_sm_b = pickle.load(open("./smPositionsRBSP_B.pickle", "rb"))

vapApitch = pickle.load(open("./PitchAngleRBSP_A.pickle", "rb"))
vapBpitch = pickle.load(open("./PitchAngleRBSP_B.pickle", "rb"))

In [8]:
print(len(pts_EPOCH_a))

1596


In [9]:
import ghostpy.algorithms.common as algc
import ghostpy.algorithms.DipoleField as dpf
import ghostpy.Invariants.FieldLine as fl
import ghostpy.Invariants.LShell as ls
import ghostpy.data.DipoleData as dpd
import ghostpy.data.VtkData as vdt

In [ ]:

ptr = pts_sm_a
data = vdt.VtkData(filename="../unit_tests/test_data/lfm_dipole_test_quad.vts", vector="B")

dL = []
dK = []
dB = []
for idx in range(len(ptr)):
    pt = ptr[idx]
    al = vapApitch[idx]
    lsx = ls.LShell(data=data, start_loc=pt, alpha=None, pre_converge=False)
    dK.append(lsx.k)
    dB.append(lsx.b)
    dL.append(lsx.l_star(res=1000))
    print ("LStar: {}".format(dL[-1]))






VVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVV
Calculating L*, K for point [-2.1490914   3.52324791  0.29740812]
K-list: [ 0.23335173  1.55172573  1.66758598]
B_mirror = 448.300157685
K = 1.6675859839
Start Phi: 2.11851775884
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
LStar: 4.15846320858



VVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVV

/usr/local/lib/python2.7/site-packages/scipy/linalg/basic.py:884: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  warnings.warn(mesg, RuntimeWarning)
/Users/jjm390/src/ghostpy/ghostpy/ghostpy/Invariants/FieldLine.py:689: RuntimeWarning: All-NaN slice encountered
  n_k = np.nanmax([kf,kb], axis=0)



Calculating L*, K for point [-2.15737571  3.49546745  0.29269665]
K-list: [ 0.          1.30545171  1.40115175]
B_mirror = 460.343185899
K = 1.40115174748
Start Phi: 2.12376142732
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
LStar: 4.13733945528



VVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVV
Calculating L*, K for point [-2.16552734  3.46746479  0.28797967]
K-list: [ 0.          0.45490386  0.49533455]
B_mirror = 458.81527177
K = 0.495334545706
Start Phi: 2.12905583983
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
LStar: 4.11799028622



VVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVV
Calculating L*, K for point [-2.17354435  3.43923474  0.28325713]
K-list: [ 0.          0.73558359  0.76804667]
B_mirror = 471.211103117
K = 0.768046668649
Start Phi: 2.13440271539
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
LStar: 4.09751452491



VVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVV
Calculating L*, K for point [-2.1814244   3.41077521 




VVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVV
Calculating L*, K for point [-2.32226195  2.65323549  0.1638525 ]
K-list: [ 0.          0.3949699   0.52187414]
B_mirror = 730.864051141
K = 0.52187413614
Start Phi: 2.28977165393
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
LStar: 3.53713941228



VVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVV
Calculating L*, K for point [-2.32561414  2.61844561  0.15904511]
K-list: [ 0.          0.76857874  0.97294379]
B_mirror = 750.328883272
K = 0.972943791074
Start Phi: 2.29703465486
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
LStar: 3.51267460829



VVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVV
Calculating L*, K for point [-2.32872987  2.58338145  0.15423733]
K-list: [ 0.00376914  0.58696783  0.74848307]
B_mirror = 762.871425643
K = 0.748483074004
Start Phi: 2.30439928466
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
LStar: 3.48807698292



VVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVV

K-list: [ 0.]
B_mirror = 1528.73404774
K = 0.0
Start Phi: 2.556514879
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
LStar: 2.75738788393



VVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVV
Calculating L*, K for point [-2.29180372  1.47890641  0.0208593 ]
K-list: [ 0.]
B_mirror = 1576.65739737
K = 0.0
Start Phi: 2.56852688959
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
LStar: 2.72762104914



VVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVV
Calculating L*, K for point [-2.28466313  1.43488157  0.01620782]
K-list: [ 0.]
B_mirror = 1630.54359764
K = 0.0
Start Phi: 2.58080343247
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
LStar: 2.69804753069



VVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVV
Calculating L*, K for point [-2.2770042   1.39052745  0.0115704 ]
K-list: [ 0.03060104  0.15922809  0.91468009]
B_mirror = 1691.23602286
K = 0.914680085048
Start Phi: 2.59335511185
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
LStar: 2.66810886017



...

VVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVV
Calculating L*, K for point [ 2.24351005  4.15281961  1.12638361]
K-list: [ 12.38817693  14.9514334   15.14338384]
B_mirror = 296.121900936
K = 15.1433838414
Start Phi: 1.07547898347
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
LStar: 5.1273104242



VVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVV
Calculating L*, K for point [ 2.23452734  4.17375304  1.12667173]
K-list: [ 10.56870083  15.12868277  15.17473811]
B_mirror = 293.928607588
K = 15.1747381132
Start Phi: 1.07925001393
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
LStar: 5.14041191105



VVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVV
Calculating L*, K for point [ 2.22546087  4.19452187  1.12691964]
K-list: [ 11.39738147  15.16276158  15.36335458]
B_mirror = 293.557153979
K = 15.363354577
Start Phi: 1.08299780098
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
LStar: 5.15340280403



VVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVV

VVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVV
Calculating L*, K for point [ 1.98311232  4.64707044  1.12190279]
K-list: [  9.34878872  11.9993741   12.06234538]
B_mirror = 240.162480418
K = 12.0623453757
Start Phi: 1.16744889781
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
LStar: 5.4286775974



VVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVV
Calculating L*, K for point [ 1.97220365  4.66400794  1.12126524]
K-list: [  7.11407728  12.05156207  12.0599976 ]
B_mirror = 238.202346544
K = 12.0599975993
Start Phi: 1.17074308391
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
LStar: 5.43868881454



VVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVV
Calculating L*, K for point [ 1.96123086  4.68079967  1.12059575]
K-list: [  8.50529141  12.55027191  12.59491978]
B_mirror = 240.024667613
K = 12.5949197809
Start Phi: 1.17402308784
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
LStar: 5.4486289611



VVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVV

LStar: 5.6462305627



VVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVV
Calculating L*, K for point [ 1.67898159  5.04117235  1.09537398]
K-list: [  8.12656899  10.17629408  10.25724641]
B_mirror = 207.436015286
K = 10.2572464103
Start Phi: 1.24929737709
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
LStar: 5.65373743722



VVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVV
Calculating L*, K for point [ 1.66624787  5.05452903  1.09396308]
K-list: [  8.2044601   11.10503087  11.1054884 ]
B_mirror = 209.118397875
K = 11.1054884004
Start Phi: 1.25236043518
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
LStar: 5.66096650826



VVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVV
Calculating L*, K for point [ 1.6538289   5.06761637  1.09252234]
K-list: [  5.29589753   9.7073265   10.26147887]
B_mirror = 205.563050066
K = 10.2614788689
Start Phi: 1.25534169124
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
LStar: 5.66820330592



VVVVVVVVVVVVVVVVVVVVVVV

VVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVV
Calculating L*, K for point [ 1.32727251  5.35097096  1.04807719]
K-list: [  9.13475486  11.88908263  11.99898624]
B_mirror = 198.350411349
K = 11.9989862418
Start Phi: 1.32766010807
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
LStar: 5.81264835761



VVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVV
Calculating L*, K for point [ 1.31335158  5.36063129  1.04597914]
K-list: [ 6.77255058  9.21588336  9.22718762]
B_mirror = 187.384580406
K = 9.2271876173
Start Phi: 1.33052959745
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
LStar: 5.81714354663



VVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVV
Calculating L*, K for point [ 1.29977758  5.37006511  1.04385811]
K-list: [ 6.5045352   8.77298822  8.97168412]
B_mirror = 186.559119236
K = 8.97168412178
Start Phi: 1.33332211074
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
LStar: 5.82143955932



VVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVV

LStar: 5.89839077193



VVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVV
Calculating L*, K for point [ 0.94776137  5.56317168  0.98363076]
K-list: [ 4.22241379  6.71482318  6.89715615]
B_mirror = 173.113054224
K = 6.89715614635
Start Phi: 1.40205290541
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
LStar: 5.90029575783



VVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVV
Calculating L*, K for point [ 0.93334066  5.56916598  0.98094867]
K-list: [ 3.63831543  6.58632104  7.02365019]
B_mirror = 174.172460551
K = 7.02365019011
Start Phi: 1.40474868406
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
LStar: 5.90188772958



VVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVV
Calculating L*, K for point [ 0.9184907   5.57509778  0.97824721]
K-list: [ 3.46075357  6.31157596  6.37711222]
B_mirror = 171.661923958
K = 6.37711221747
Start Phi: 1.40751425542
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
LStar: 5.90370870759



VVVVVVVVVVVVVVVVVVVVVVVVVVVVVVV

In [ ]:
ptr = pts_sm_b

dL2 = []
dK2 = []
dB2 = []
for idx in range(len(ptr)):
    pt = ptr[idx]
    al = vapBpitch[idx]
    lsx = ls.LShell(data=data, start_loc=pt, alpha=al, pre_converge=False)
    dK2.append(lsx.k)
    dB2.append(lsx.b)
    dL2.append(lsx.l_star(res=1000))
    print ("LStar: {}".format(dL2[-1]))







VVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVV
Calculating L*, K for point [ 1.84636003 -0.18303698  0.18400538]
Invalid line initiation
Starting Location: [ 1.84636003 -0.18303698  0.18400538]
LStar: nan



VVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVV
Calculating L*, K for point [ 1.88275019 -0.1385805   0.1941237 ]
Invalid line initiation
Starting Location: [ 1.88275019 -0.1385805   0.1941237 ]
LStar: nan



VVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVV
Calculating L*, K for point [ 1.9179742  -0.09388928  0.20405699]
Invalid line initiation
Starting Location: [ 1.9179742  -0.09388928  0.20405699]
LStar: nan



VVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVV
Calculating L*, K for point [ 1.95204201 -0.0491296   0.21380385]
Invalid line initiation
Starting Location: [ 1.95204201 -0.0491296   0.21380385]
LStar: nan



VVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVV
Calculating L*, K for point 

K: nan
^^^^^^^^^^^^^^^^^^^^^^^^
Start Phi: 0.341036730214
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
LStar: nan



VVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVV
Calculating L*, K for point [ 2.47101666  0.91340675  0.38519853]
K-list: []
B_mirror = 24444.2817427
K = nan
vvvvvvvvvvvvvvvvvvvvvvvv
INITIALIZATION OF LINE
Origin: [ 2.47101666  0.91340675  0.38519853]
B: 24444.2817427
K: nan
^^^^^^^^^^^^^^^^^^^^^^^^
Start Phi: 0.354070407653
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
LStar: nan



VVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVV
Calculating L*, K for point [ 2.48638697  0.95529731  0.39126157]
K-list: []
B_mirror = 4840.78009459
K = nan
vvvvvvvvvvvvvvvvvvvvvvvv
INITIALIZATION OF LINE
Origin: [ 2.48638697  0.95529731  0.39126157]
B: 4840.78009459
K: nan
^^^^^^^^^^^^^^^^^^^^^^^^
Start Phi: 0.366821541911
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
LStar: nan



VVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVV
Calculati

LStar: 3.42528891369



VVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVV
Calculating L*, K for point [ 2.72043066  1.93005733  0.50499836]
K-list: [ 24.96123274  33.82389989  33.82873447  38.08446566]
B_mirror = 1151.80752367
K = 38.0844656566
Start Phi: 0.617051817446
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
LStar: 3.45073118842



VVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVV
Calculating L*, K for point [ 2.72485832  1.96625385  0.50823096]
K-list: [  94.18818848  117.69461995  117.80852562  118.41487356]
B_mirror = 2225.75665072
K = 118.414873565
Start Phi: 0.625075050649
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
LStar: 3.47567553162



VVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVV
Calculating L*, K for point [ 2.72897788  2.00223867  0.51137622]
K-list: []
B_mirror = 9053.31436272
K = nan
vvvvvvvvvvvvvvvvvvvvvvvv
INITIALIZATION OF LINE
Origin: [ 2.72897788  2.00223867  0.51137622]
B: 9053.31436272
K: nan
^^^^^^^^^^^^^^^

Calculating L*, K for point [ 2.75329289  2.73960545  0.56036882]
K-list: []
B_mirror = 9788.20061687
K = nan
vvvvvvvvvvvvvvvvvvvvvvvv
INITIALIZATION OF LINE
Origin: [ 2.75329289  2.73960545  0.56036882]
B: 9788.20061687
K: nan
^^^^^^^^^^^^^^^^^^^^^^^^
Start Phi: 0.782906327184
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
LStar: nan



VVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVV
Calculating L*, K for point [ 2.75167541  2.77087355  0.56176884]
K-list: []
B_mirror = 5546.460086
K = nan
vvvvvvvvvvvvvvvvvvvvvvvv
INITIALIZATION OF LINE
Origin: [ 2.75167541  2.77087355  0.56176884]
B: 5546.460086
K: nan
^^^^^^^^^^^^^^^^^^^^^^^^
Start Phi: 0.78887446895
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
LStar: nan



VVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVV
Calculating L*, K for point [ 2.75005865  2.80172089  0.56309616]
K-list: [  83.19754284   99.27398741   99.49413262  102.63667232]
B_mirror = 1308.46094361
K = 102.636672324
Start Phi: 0.794703422

Calculating L*, K for point [ 2.65441545  3.48937537  0.57717852]
K-list: [  6.56284487  10.3085491   10.56542041  12.56808396]
B_mirror = 408.088863912
K = 12.5680839644
Start Phi: 0.920473671874
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
LStar: 4.49686799084



VVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVV
Calculating L*, K for point [ 2.64838473  3.51582436  0.5770893 ]
K-list: [ 21.84448824  26.5376792   26.58585573  29.20841521]
B_mirror = 498.047064108
K = 29.208415206
Start Phi: 0.925202123044
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
LStar: 4.51451722247



VVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVV
Calculating L*, K for point [ 2.64247229  3.54190769  0.57695133]
K-list: [  87.93282476  100.04533249  100.12757248  103.45510156]
B_mirror = 976.113222192
K = 103.455101559
Start Phi: 0.929822200799
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
LStar: 4.5313182685



VVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVV
Ca

...

LStar: 4.70819294017



VVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVV
Calculating L*, K for point [ 2.47258291  3.43558767  1.16944791]
K-list: [ 152.46874565  162.72762511  162.79123052  173.88286741]
B_mirror = 1312.78790392
K = 173.882867407
Start Phi: 0.946972706608
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
LStar: 4.72534127157



VVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVV
Calculating L*, K for point [ 2.46691804  3.46141825  1.17117578]
K-list: [ nan   0.]
B_mirror = 4112.70765876
K = nan
vvvvvvvvvvvvvvvvvvvvvvvv
INITIALIZATION OF LINE
Origin: [ 2.46691804  3.46141825  1.17117578]
B: 4112.70765876
K: nan
^^^^^^^^^^^^^^^^^^^^^^^^
Start Phi: 0.951604284746
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
LStar: nan



VVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVV
Calculating L*, K for point [ 2.46113137  3.48705328  1.17285971]
K-list: []
B_mirror = 13532.4066054
K = nan
vvvvvvvvvvvvvvvvvvvvvvvv
INITIALIZATION OF LINE
Orig

^^^^^^^^^^^^^^^^^^^^^^^^
Start Phi: 1.04431504048
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
LStar: nan



VVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVV
Calculating L*, K for point [ 2.30753851  4.00616817  1.20049764]
K-list: []
B_mirror = 8785.18987133
K = nan
vvvvvvvvvvvvvvvvvvvvvvvv
INITIALIZATION OF LINE
Origin: [ 2.30753851  4.00616817  1.20049764]
B: 8785.18987133
K: nan
^^^^^^^^^^^^^^^^^^^^^^^^
Start Phi: 1.04821353594
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
LStar: nan



VVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVV
Calculating L*, K for point [ 2.2993231   4.02789857  1.2013568 ]
K-list: [ 219.39792797  241.3716086   241.45835858  250.21382002]
B_mirror = 1619.53836303
K = 250.213820023
Start Phi: 1.05208699812
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
LStar: 5.10961869555



VVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVV
Calculating L*, K for point [ 2.29130863  4.04930434  1.2021842 ]
K-list: [  89.4082663




VVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVV
Calculating L*, K for point [ 2.08012412  4.50116351  1.21286001]
K-list: [ 55.11810898  63.6793632   63.8732354   63.94076677]
B_mirror = 448.97961259
K = 63.940766772
Start Phi: 1.13790074184
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
LStar: 5.40813193686



VVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVV
Calculating L*, K for point [ 2.06997371  4.51902684  1.21297898]
K-list: [ 64.40595923  76.7011701   76.95222117  77.06353095]
B_mirror = 503.571872712
K = 77.0635309477
Start Phi: 1.14126209788
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
LStar: 5.41912967208



VVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVV
Calculating L*, K for point [ 2.05975664  4.53674432  1.21307118]
K-list: [ 139.79244796  151.519758    151.7816394   160.73090306]
B_mirror = 867.027928423
K = 160.730903061
Start Phi: 1.14460732714
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
LStar: 5.43002228924



VV

K-list: [ 156.96222111  166.79761509  167.36807849  177.66099211]
B_mirror = 863.307353013
K = 177.660992108
Start Phi: 1.21407131838
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
LStar: 5.6412350948



VVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVV
Calculating L*, K for point [ 1.81051222  4.90366835  1.20802399]
K-list: [   0.          523.35750149]
B_mirror = 4047.51800433
K = 523.357501494
Start Phi: 1.21710628548
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
LStar: 5.64969835972



VVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVV
Calculating L*, K for point [ 1.79926783  4.91785029  1.20750436]
K-list: []
B_mirror = 10583.1510837
K = nan
vvvvvvvvvvvvvvvvvvvvvvvv
INITIALIZATION OF LINE
Origin: [ 1.79926783  4.91785029  1.20750436]
B: 10583.1510837
K: nan
^^^^^^^^^^^^^^^^^^^^^^^^
Start Phi: 1.22005863872
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
LStar: nan



VVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVV
Calculating L*, K for 

B_mirror = 14722.5636244
K = nan
vvvvvvvvvvvvvvvvvvvvvvvv
INITIALIZATION OF LINE
Origin: [ 1.53465895  5.19763707  1.19009396]
B: 14722.5636244
K: nan
^^^^^^^^^^^^^^^^^^^^^^^^
Start Phi: 1.28369299096
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
LStar: nan



VVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVV
Calculating L*, K for point [ 1.52241969  5.20873862  1.1890375 ]
K-list: [ 199.39846747  213.74397848  213.93112384  218.46282082]
B_mirror = 1027.95472543
K = 218.462820818
Start Phi: 1.28643537056
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
LStar: 5.82022798901



VVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVV
Calculating L*, K for point [ 1.50976878  5.21982028  1.18795901]
K-list: [ 73.29149312  81.83942202  82.24819413  87.07039939]
B_mirror = 435.584326278
K = 87.0703993943
Start Phi: 1.28924221699
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
LStar: 5.82609837619



VVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVV
Calculat

In [ ]:
import cPickle as pickle

pickle.dump(dL, open("fdLA.pickle", "wb"))
pickle.dump(dK, open("fdKA.pickle", "wb"))
pickle.dump(dB, open("fdBA.pickle", "wb"))


pickle.dump(dL2, open("fdLB.pickle", "wb"))
pickle.dump(dK2, open("fdKB.pickle", "wb"))
pickle.dump(dB2, open("fdBB.pickle", "wb"))

pickle.dump(pts_EPOCH_a, open("fEpochA.pickle", "wb"))
pickle.dump(pts_EPOCH_b, open("fEpochB.pickle", "wb"))

pickle.dump(pts_sm_a, open("fCoordSMA.pickle", "wb"))
pickle.dump(pts_sm_b, open("fCoordSMB.pickle", "wb"))



